# Convolutional Neural Networks - Image Classification [using Pytorch]

### 1. INITIALIZATION

In [1]:
# Import necessary libraries and modules
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Check tensorflow version
torch.__version__

'2.2.2'

### 2. LOADING DATASET

The dataset used is [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), which is a widely used benchmark dataset in the field of computer vision and machine learning. It consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. It serves as a standard dataset for training and evaluating machine learning algorithms, particularly for image classification tasks.

The classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck.

Before loading the dataset, we need to initialize data augmentation and image transformations to avoid over fitting. This helps to increase the diversity of the dataset and improve the generalization ability of the CNN model.

Transformations of training images are:
- Randomly rotate images
- Center crop to the images
- Randomly flip images horizontally
- Randomly changing the brightness, contrast, saturation, and hue of the image (color jitter)
- Scale the pixel values of the images to be in the range [0, 1] (automatically done when converting to tensor)

In [ ]:
# Creating transformer as a series of steps
transform_training = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.CenterCrop(24),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
# Loading the dataset from pytorch library
data_train = torchvision.datasets.CIFAR10(root="../data/raw", train=True, download=True, transform=None)
data_test = torchvision.datasets.CIFAR10(root="../data/raw", train=False, download=True, transform=None)

100.0%


Extracting ../data/raw\cifar-10-python.tar.gz to ../data/raw
Files already downloaded and verified


In [ ]:
# Initialize data loaders
train_loader = torch.utils.data.DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=32, shuffle=False)

### 3. IMAGE PREPROCESSING

### 4. BUILDING CNN MODEL

### 5. TRAINING CNN MODEL

### 6. PERFORMANCE ANALYSIS